In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import re
import struct
import binascii
from crccheck.crc import Crc8, Crc8Maxim
from crccheck.checksum import Checksum8

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
def getCRC(data):
    crcinst = Crc8Maxim()
    crcinst.process(data)
    crchex = crcinst.finalhex().upper()
    return crchex

def getInt(data):
    encodedData = data.encode()
    encodedDataLen = len(encodedData)
    
    # @ Native
    # = Standard
    # < Little Endian
    # > Big Endian
    # ! Network
    s = struct.Struct('< ' + str(encodedDataLen) + 's')
    packedData = s.pack(encodedData)
    
    f = lambda n:n.to_bytes(int(encodedDataLen/2), "little").hex()
    
    lsbVal = f(int(packedData, 16))
    
    return int(lsbVal, 16)

def getFahrenheit(c):
    return round((c * (9/5)) + 32, 2)

def getColFromRow(row, namedData):
    hexStrVal = ''.join([row[cols[namedData][0]+x] for x in range(0, cols[namedData][1])])
    return hexStrVal;



def loadFromBin(df, fileUri):
    file = open(fileUri, "rb")

    byteRead = file.read(36)
    while byteRead:
        crc = getCRC(byteRead)

        if crc == '00':
            row = (''.join(format(x, '02x') for x in byteRead)).upper()

            splitRow = [row[i:i+2] for i in range(0, len(row), 2)]
            
            data = {}            
            for i in range(0, len(splitRow)):
                data['C' + str(i+1)] = splitRow[i+0]

            df = df.append(data, ignore_index=True)

        else:
            print("Bad CRC")
            display(byteRead)

        byteRead = file.read(36)

    file.close()
    
    return df;

def parseData(df, dfData):
    df = df.iloc[0:0]
    
    for i, row in enumerate(dfData.itertuples(), 1):
        stateA = getInt(getColFromRow(row, "stateA"))

        energyGauge = getInt(getColFromRow(row, "energyGauge"))
        unknownA = getInt(getColFromRow(row, "unknownA"))

        tempA = getInt(getColFromRow(row, "tempA"))
        tempB = getInt(getColFromRow(row, "tempB"))
        tempC = getInt(getColFromRow(row, "tempC"))
        tempD = getInt(getColFromRow(row, "tempD"))
        tempFA = getFahrenheit(tempA)
        tempFB = getFahrenheit(tempB)
        tempFC = getFahrenheit(tempC)
        tempFD = getFahrenheit(tempD)

        unknownB = getInt(getColFromRow(row, "unknownB"))

        totalVoltage = round(getInt(getColFromRow(row, "totalVoltage")) / 1000, 2)
        totalCurrent = round(getInt(getColFromRow(row, "totalCurrent")) / 1000, 2)
        cellVoltageHigh = round(getInt(getColFromRow(row, "cellVoltageHigh")) / 1000, 2)
        cellVoltageLow = round(getInt(getColFromRow(row, "cellVoltageLow")) / 1000, 2)
        chargingState = 'ON' if getInt(getColFromRow(row, "chargingState")) < 0xFF else 'OFF'
        
        unknownC = round(getInt(getColFromRow(row, "unknownC")) / 1000, 2)

        df = df.append({
            'stateA':stateA,
###############
# New Insight #
###############
            'energyGauge':energyGauge,
###############
            'unknownA':unknownA,
            'tempA':tempA,
            'tempB':tempB,
            'tempC':tempC,
            'tempD':tempD,
            'tempFA':tempFA,
            'tempFB':tempFB,
            'tempFC':tempFC,
            'tempFD':tempFD,
            'unknownB':unknownB,
            'totalVoltage':totalVoltage,
            'totalCurrent':totalCurrent,
            'cellVoltageHigh':cellVoltageHigh,
            'cellVoltageLow':cellVoltageLow,
###############
# New Insight #
###############
            'chargingState':chargingState,
###############
            'unknownC':unknownC,
        }, ignore_index=True)

    return df;

In [ ]:
cols = {
    "stateA": [4, 1, '#80FF00'],
    
    "energyGauge": [6, 1, '#CC66CC'], ## 0x00-0x64 (0-100 %)
    "unknownA": [7, 1, '#FF99FF'],

    "tempA": [8, 1, '#CCFFCC'],
    "tempB": [9, 1, '#CCFFCC'],
    "tempC": [10, 1, '#CCFFCC'],
    "tempD": [11, 1, '#CCFFCC'],

    "unknownB": [12, 1, '#FF99FF'],

    "totalVoltage": [22, 2, '#CCFFCC'],
    "totalCurrent": [26, 2, '#FFCCCC'],
    "chargingState": [28, 2, '#CCCCCFF'],
    "cellVoltageHigh": [30, 2, '#AAFFAA'],
    "cellVoltageLow": [32, 2, '#FFAAAA'],

    "unknownC": [34, 2, '#FF99FF'],

    "crc": [36, 1, '#AAAAAA'],
}

colsSizes = {}
for idx, val in enumerate(cols):
    colsSizes[val] = [cols[val][0], cols[val][0]+cols[val][1]-1]

In [ ]:
dfData = pd.DataFrame([], columns=[
    'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10',
    'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
    'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30',
    'C31', 'C32', 'C33', 'C34', 'C35', 'C36'
])

dfProcessed = pd.DataFrame([], columns=[
    'stateA',

    'energyGauge',
    'unknownA',
    
    'tempA',
    'tempB',
    'tempC',
    'tempD',
    'tempFA',
    'tempFB',
    'tempFC',
    'tempFD',

    'unknownB',
    
    'totalVoltage',
    'totalCurrent',
    'cellVoltageHigh',
    'cellVoltageLow',

    'chargingState',
    
    'unknownC',
])

In [ ]:
dfData.iloc[0:0]
dfData = loadFromBin(dfData, fileUri = "../Data/sample1.bin") # Discharging
dfData = loadFromBin(dfData, fileUri = "../Data/sample2.bin") # Discharging
dfData = loadFromBin(dfData, fileUri = "../Data/sample3.bin") # Discharging
dfData = loadFromBin(dfData, fileUri = "../Data/sample4.bin") # Discharging
dfData = loadFromBin(dfData, fileUri = "../Data/sample5.bin") # Depleated battery BMS shut down
dfData = loadFromBin(dfData, fileUri = "../Data/sample6.bin") # Charging -> Charged

In [ ]:
%%capture

def colorColumn(dat, c='red'):
    return [f'background-color: {c}' for i in dat]

style = dfData.style
for idx, val in enumerate(cols):
    subset = ['C'+str(cols[val][0]+k) for k in range(0, cols[val][1])]
    style = style.apply(colorColumn, axis=0, subset=subset, c=cols[val][2])

# Determin if there was a data change from the previous column value:
# - Ignore the first row (0)
# - Ignore the last column (C36)
style.apply(lambda x: [
    "background-color: #FFFFBD;border:2px solid #353500;" if (i > 0 and x.name != 'C36' and (v != x.iloc[i-1])) 
    else "" for i, v in enumerate(x)], axis = 0)

In [ ]:
display(style)

In [ ]:
dfProcessed = parseData(dfProcessed, dfData)
display(dfProcessed)

In [ ]:
startChargeIdx = dfData.loc[dfData['C29'] == '00'].head(1).index.astype(int)[0]
display(startChargeIdx)

endChargeIdx = dfData.loc[(dfData['C6'] == '64') & (dfData['C20'] != '00') & (dfData['C29'] == 'FF')].head(1).index.astype(int)[0]
display(endChargeIdx)

ax = dfProcessed.plot(y='energyGauge', kind='line', colormap='RdYlGn')
ax.annotate('Start Charge', xy=(startChargeIdx, 0), xytext=(startChargeIdx, -20), arrowprops=dict(arrowstyle="->", connectionstyle="angle3,angleA=0,angleB=-90"))
ax.annotate('Full Charge', xy=(endChargeIdx, 0), xytext=(endChargeIdx, -20), arrowprops=dict(arrowstyle="->", connectionstyle="angle3,angleA=0,angleB=-90"))
dfProcessed.plot(ax=ax, y='totalVoltage', secondary_y=True, kind='line')
plt.show()

ax = dfProcessed.plot(y='energyGauge', kind='line', colormap='RdYlGn')
ax.annotate('Start Charge', xy=(startChargeIdx, 0), xytext=(startChargeIdx, -20), arrowprops=dict(arrowstyle="->", connectionstyle="angle3,angleA=0,angleB=-90"))
ax.annotate('Full Charge', xy=(endChargeIdx, 0), xytext=(endChargeIdx, -20), arrowprops=dict(arrowstyle="->", connectionstyle="angle3,angleA=0,angleB=-90"))
dfProcessed.plot(ax=ax, y='totalCurrent', secondary_y=True, kind='line')
plt.show()

ax = dfProcessed.plot(y=['tempA', 'tempB', 'tempC', 'tempD'], kind = 'line', colormap='PiYG')
plt.show()

ax = dfProcessed.plot(y=['tempFA', 'tempFB', 'tempFC', 'tempFD'], kind = 'line', colormap='PiYG')
plt.show()

color = {
    "boxes": "DarkGreen",
    "whiskers": "DarkOrange",
    "medians": "DarkBlue",
    "caps": "Gray",
}
dfProcessed.boxplot(column=["cellVoltageHigh", "cellVoltageLow"], by=["chargingState"]);
plt.show()

ax = dfProcessed.plot(y=['cellVoltageHigh', 'cellVoltageLow'], kind = 'line')
plt.show()

### Exploring Values—Converted to Fraction and Integer

In [ ]:
testValue = '522C'
display(round(getInt(testValue) / 1000, 2))

testValue = '52'
display(round(getInt(testValue), 2))

### Distinct Column Values

In [ ]:
display(dfData['C5'].unique())